In [29]:
import pandas as pd
import numpy as np

In [30]:
# func 모음

# 1. 종목코드를 6자리로 통일. ex) 990 -> 000990
def padding(x):
    x = str(x)
    return x.rjust(6, "0")

# 2. 결산월 12월인 것만 선택
# : 전체 데이터 4963개 중에서 12월 결산인 데이터가 4836개이다. 나머지 결산월의 데이터가 전체의 2.5%로 데이터량이 작기도하고 결측치가 많아 제거
def func_yearin(x):
    if x[5:7] == '12':
        return True
    else:
        return False

# 3. 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

# 4. 연결, 개별재무제표 통합을 위해 연결재무 데이터 컬럼명 통일화 : "(IFRS연결)"제거
def func_replace_col1(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS연결)","")

# 5. 연결, 개별재무제표 통합을 위해 개별재무 데이터 컬럼명 통일화 : "(IFRS)"제거
def func_replace_col2(x):
    x = x.replace(" ","")
    x = x.replace(",","")
    x = x.replace("(비교)","")
    x = x.replace("(IFRS)","")
    x = x.replace("(*)","")
    x = x.replace("[제조]","")
    return x.replace("(IFRS)","")

# 6. 2020년 raw data 제외
def func_rm_2020(x):
    if x[:4] == "2020":
        return False
    else:
        return True

# 7. True, False 반전
def func_not(x):
    return not x

In [31]:
개별토빈 = pd.read_csv('./datasets/raw/개별토빈(주식의종류10).csv',encoding='cp949')
연결토빈 = pd.read_csv('./datasets/raw/연결토빈(주식의종류10).csv',encoding='cp949')

개별토빈["거래소코드"] = 개별토빈["거래소코드"].map(padding) # 거래소코드 padding
개별토빈["회계년도"] = 개별토빈["회계년도"].map(change_year) # 년도만 표시

연결토빈["거래소코드"] = 연결토빈["거래소코드"].map(padding) # 거래소코드 padding
연결토빈["회계년도"] = 연결토빈["회계년도"].map(change_year) # 년도만 표시

# 토빈q  = (총부채 + 산술평균*발행주식의총수)/자산 

In [32]:
연결토빈.columns = 연결토빈.columns.map(func_replace_col1)
개별토빈.columns = 개별토빈.columns.map(func_replace_col2)

In [33]:
연결토빈 = 연결토빈[연결토빈['주식의종류']==10]
게별토빈 = 개별토빈[개별토빈['주식의종류']==10]

In [34]:
## 중복된 값 제거
연결토빈.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
연결토빈.reset_index(drop=True, inplace=True)

## 외국계기업제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

연결토빈 = 연결토빈[연결토빈["거래소코드"].map(func_isnt_fcompany)]

# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (연결토빈[연결토빈["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(연결토빈["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = 연결토빈[연결토빈["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
연결토빈 = pd.concat(dfs)
연결토빈.reset_index(drop=True, inplace=True)


## 기업명 변경
연결토빈[연결토빈['거래소코드']=='036420']
연결토빈.loc[연결토빈['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [35]:
## 중복된 값 제거
개별토빈.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
개별토빈.reset_index(drop=True, inplace=True)

## 외국계기업제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

개별토빈 = 개별토빈[개별토빈["거래소코드"].map(func_isnt_fcompany)]

# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (개별토빈[개별토빈["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(개별토빈["회사명"].unique())
dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = 개별토빈[개별토빈["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
개별토빈 = pd.concat(dfs)
개별토빈.reset_index(drop=True, inplace=True)


## 기업명 변경
개별토빈[개별토빈['거래소코드']=='036420']
개별토빈.loc[개별토빈['회사명']=='(주)제이콘텐트리',['회사명']] = '(주)콘텐트리중앙'

In [36]:
연결토빈.isna().sum()

회사명                              0
거래소코드                            0
회계년도                             0
자산                            4737
부채                            4737
산술평균                           181
발행주식의총수(현재발행한주식수-현재감소한주식수)       0
주식의종류                            0
dtype: int64

In [37]:
개별토빈.isna().sum()

회사명                             0
거래소코드                           0
회계년도                            0
자산                            376
부채                            376
산술평균                          200
발행주식의총수(현재발행한주식수-현재감소한주식수)     36
주식의종류                          36
dtype: int64

In [38]:
# df_재무를 완전데이터와 결측데이터로 나눈 뒤 결측데이터를 개별재무데이터로 대체시키고, df_재무로 다시 통합하면 끝!
토빈완전 =연결토빈[연결토빈["자산"].isna().map(func_not)]
# df_재무_완전

토빈결측 =연결토빈[연결토빈["자산"].isna()]
# df_재무_결측

list_na_col = ['자산','부채','산술평균','발행주식의총수(현재발행한주식수-현재감소한주식수)','주식의종류']

토빈결측 = 토빈결측.drop(list_na_col,axis=1)
토빈결측 = pd.merge(토빈결측,개별토빈[['거래소코드','회계년도']+list_na_col],on=['거래소코드','회계년도'],how='left')

토빈_df  = pd.concat([토빈완전,토빈결측],axis=0)

In [39]:
토빈_df.isna().sum()

회사명                             0
거래소코드                           0
회계년도                            0
자산                            374
부채                            374
산술평균                          188
발행주식의총수(현재발행한주식수-현재감소한주식수)      7
주식의종류                           7
dtype: int64

In [40]:
print( '결측치가 존재하는 기업의 수',len(토빈_df[토빈_df.isna().any(1)].회사명.unique()),'개')

결측치가 존재하는 기업의 수 186 개


In [41]:
## 결측치 제거
토빈_df = 토빈_df[토빈_df["자산"].isna().map(func_not)] # 위 결측치 없는 데이터로 필터링
토빈_df

토빈_df.isna().sum().to_dict()

{'회사명': 0,
 '거래소코드': 0,
 '회계년도': 0,
 '자산': 0,
 '부채': 0,
 '산술평균': 45,
 '발행주식의총수(현재발행한주식수-현재감소한주식수)': 0,
 '주식의종류': 0}

In [48]:
토빈_df.dropna(inplace=True)

In [49]:
# 토빈q  = (총부채 + 산술평균*발행주식의총수)/자산 
토빈_df['토빈Q'] = (토빈_df['부채']+토빈_df['산술평균']*토빈_df['발행주식의총수(현재발행한주식수-현재감소한주식수)'])/토빈_df['자산']
토빈_df.drop(['자산','부채','산술평균','발행주식의총수(현재발행한주식수-현재감소한주식수)','주식의종류'],axis=1,inplace=True)

In [52]:
토빈_df.reset_index(drop=True,inplace=True)

In [53]:
토빈_df.to_csv('토빈Q.csv',encoding='UTF-8-sig')